In [11]:
!pip install requests
!pip install BeautifulSoup4
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re


In [13]:

#URLを入れるリスト
urls = []
url = "http://www.hareruyamtg.com/purchase/e/eBuy-Mdn_sspd/"

#1ページ目を格納
urls.append(url)

#2ページ目から最後のページまでを格納
for i in range(2,6):
    i = str(i)
    url_page = "http://www.hareruyamtg.com/purchase/e/eBuy-Mdn_sspd" + "_p" + i
    urls.append(url_page) #URLのリスト


In [14]:
# indexがなく、columnsだけ設定されているDataFrameを作成
df_mtg0 = pd.DataFrame(index=[], columns=["Name"]) #カード名DF
tmp_text =[] #hrefリスト用空リスト

tmp_list = [] #

In [15]:
for u in urls:
    #URL取得、はれるやより
    result = requests.get(u)
    soup = BeautifulSoup(result.text, "lxml")
    tag_a = soup.find_all('a') #aタグのみ抽出したリスト
    
    tmp_list= [] #hrefリスト用空きリスト
    for x in tag_a: #aタグリストより、
        tmp = x.get('href')
        tmp = re.findall(r'purchase/g/g', tmp)
        if tmp:
            tmp_list.append(x.get('href')) #hrefリスト
        

    for x in tmp_list:
        url = u + x
        tmpResult = requests.get(url)
        tmpSoup = BeautifulSoup(tmpResult.text, "lxml")
        #カードテキスト取得
        tag_table = tmpSoup.find_all("td")
        tmp_text.append(tag_table[9].get_text())
        
        #カード名取得
        df = pd.DataFrame(tmpSoup.h2.text, index=[0], columns=["Name"])
        df_mtg0 = pd.concat([df_mtg0, df], sort= True)


In [16]:
#カード名のDF
df_mtg0 = df_mtg0.replace(["【ENG】《", r"/(.*)"], "", regex = True)
df_mtg0 = df_mtg0.drop_duplicates()
df_mtg0 = df_mtg0.reset_index(drop=True)
df_mtg0

,Name
0,ヴェールのリリアナ
1,オパールのモックス
2,精神を刻む者、ジェイス
3,魂の洞窟
4,解放された者、カーン
5,沸騰する小湖
6,タルモゴイフ
7,貴族の教主
8,瞬唱の魔道士
9,地平線の梢


In [62]:
#カードテキストのDF
tmp_text = pd.DataFrame(tmp_text, columns = ["Text"])
tmp_text = tmp_text.replace(r'\r\n\t\t\t\t','', regex=True)
df_text = tmp_text.drop_duplicates()
df_text = df_text.reset_index(drop=True)

#カード名DF、テキストDFを結合
df_cards = pd.concat([df_mtg0, df_text], axis=1)
#dataframeの表示文字数をデフォルトの50字から80字に変更
pd.set_option("display.max_colwidth", 200)
df_cards = df_cards.replace([r"[a-zA-Z]", r"\[.+\]"], "", regex=True)

#ここから、自然言語処理。まず、Mecabを使って形態素解析を行う。

In [26]:
#MeCabのダウンロード
!pip install mecab-python3
import sys
import MeCab

    100% |████████████████████████████████| 51kB 4.0MB/s ta 0:00:011
  Running setup.py bdist_wheel for mecab-python3 ... done
  Stored in directory: /Users/moriha/Library/Caches/pip/wheels/4c/07/3a/5f22ccc9f381f3bc01fa023202061cd1e0e9af855292f005dd
Successfully built mecab-python3


In [66]:
meshi_list = []
name = MeCab.Tagger()
words = name.parse(df_cards.iat[0, 1])
for num in words.split("\n"):
    word = num.split("\t")[0]
    if word == "EOS":
        break
    else:
        meshi_list.append(word)
print(meshi_list)

['：', 'プレイヤー', '１', '人', 'を', '対象', 'と', 'し', '、', 'その', 'プレイヤー', 'が', 'コントロール', 'する', 'すべて', 'の', 'パーマネント', 'を', '２つ', 'の', '束', 'に', '分ける', '。', 'その', 'プレイヤー', 'は', '束', 'を', '１つ', '選び', '、', 'その', '束', 'に', 'ある', 'すべて', 'の', 'パーマネント', 'を', '生け贄', 'に', '捧げる', '。']
